In [3]:
#Mineral needs for dairy cows following NRC 2001
name = input('What is your name? \n')
import math

What is your name? 
 Aaron


In [4]:
import csv
with open('info.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
    writer.writerow(['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain'])

In [12]:
#Enter basic information for the calculation
def WriteNewRow():
    cowID = input('Cow ID#: ')
    bw = input('Body weight (lb, use whole numbers): ')
    mw = float(bw)**0.75
    dim = input('Days lactating? If not a lactating cow, type "0".\n')
    dp = input('Days pregnant? If not pregnant, type "0".\n')
    breed = input('What is the breed? Type "holstein", "jersey", or "other".')
    milkprod = input('Milk yield per day (kg/day): \n')
    milkfat = input('Percent milk fat: \n')
    wg = input('Average net bodyweight gain per day (lb/day): \n')
    NewRow = cowID,bw,mw,dim,dp,breed,milkprod,milkfat,wg
    import csv
    from csv import writer
    with open('info.csv', 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(NewRow)
    import csv
    with open('info.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            print(row)

In [13]:
WriteNewRow()

Cow ID#:  1111
Body weight (lb, use whole numbers):  500
Days lactating? If not a lactating cow, type "0".
 3
Days pregnant? If not pregnant, type "0".
 200
What is the breed? Type "holstein", "jersey", or "other". holstein
Milk yield per day (kg/day): 
 50
Percent milk fat: 
 3
Average net bodyweight gain per day (lb/day): 
 .7


['Cow ID', 'Body weight', 'mw', 'Days in milking', 'Days of pregnancy', 'Breed', 'Milk production', 'Milk fat', 'Weight gain']
['1111', '500', '105.7371263440564', '3', '200', 'holstein', '50', '10', '.5']
['1111', '500', '105.7371263440564', '3', '200', 'holstein', '50', '3', '.7']


In [14]:
def AddCowsQ():
    AddCows = input('Add another cow? (Y/N) \n')
    if AddCows == "Y":
        WriteNewRow()
        AddCowsQ()
    else:
        print('Ready to calculate Ca and P needs!')

In [15]:
AddCowsQ()

Add another cow? (Y/N) 
 N


Ready to calculate Ca and P needs!


In [8]:
#Below we will append Ca and P needs for each cow in new columns using Pandas

In [85]:
import pandas as pd
import numpy as np
import math
columns = ['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain']
Cows = pd.read_csv("./info.csv", usecols=columns)

###Ca Needs

#Urine Ca
try:
    Cows['CaUrine'] = 0.08*(Cows['Body weight'] / 100)
except:
    Cows['CaUrine'] = 'error'

#Fecal Ca
try:
    if ((Cows['Days in milking']) > 0).all():
        Cows['CaFecal'] = 3.1*(Cows['Body weight'] / 100)
    else:
        Cows['CaFecal'] = 1.54*(Cows['Body weight'] / 100)
except:
    Cows['CaFecal'] = 'error'

#Fetal Ca
try:
    if ((Cows['Days of pregnancy']) > 189).all():
        Cows['CaFetal'] = 0.02456*np.exp((0.05581-(0.00007*(Cows['Days of pregnancy'])))*(Cows['Days of pregnancy']))-0.02456*np.exp((0.05581-(0.00007*((Cows['Days of pregnancy'])-1)))*((Cows['Days of pregnancy'])-1))
    else:
        Cows['CaFetal'] = 0
except:
    Cows['CaFetal'] = 'error'

#Lactation Ca
try:
    if (Cows['Breed']).any() == 'holstein':
        Cows['CaMilkProd'] = 1.22*(Cows['Milk production'])
    elif (Cows['Breed']).any() == 'jersey':
        Cows['CaMilkProd'] = 1.45*(Cows['Milk production'])
    else:
        Cows['CaMilkProd'] = 1.37*(Cows['Milk production'])
    #print('Lactation =', round(float(CaMilkProd), 2))
except:
    Cows['CaMilkProd'] = 'error'

#Growing Ca
try:
    if ((Cows['Weight gain']) > 0).all():
        Cows['CaGrowing'] = (9.83*((Cows['mw'])**0.22)*(Cows['Body weight'])**-0.22)*((Cows['Weight gain'])/0.96)
    else:
        Cows['CaGrowing'] = 0
    #print('growing =', round(float(CaGrowing), 2))
except:
    Cows['CaGrowing'] = 'error'
       
Cows['Total Ca Need'] = Cows['CaUrine'] + Cows['CaFecal'] + Cows['CaFetal'] + Cows['CaMilkProd'] + Cows['CaGrowing']

#Urine P
Cows['PUrine'] = 0.002*(Cows['Body weight'])
#print('urinary =', round(float(PUrine), 2))

#Fecal P
FCM = (0.4*(Cows['Milk production']))+(15*(Cows['Milk fat'] / 100)*Cows['Milk production'])
DMI = ((0.372*(FCM))+(0.0968*(Cows['mw'])))*(1-np.exp(-0.192*((Cows['Days in milking'] / 7)+3.67)))
Cows['PFecal'] = (DMI)

###P Needs

#Fetal P
try:
    if (Cows['Days of pregnancy'] > 189).all():
        Cows['PFetal'] = 0.02743*np.exp((0.05527-(0.000075*(Cows['Days of pregnancy'])))*(Cows['Days of pregnancy']))-0.02743*np.exp((0.05527-(0.000075*((Cows['Days of pregnancy'])-1)))*((Cows['Days of pregnancy'])-1))
    else:
        Cows['PFetal'] = 0
except:
    Cows['PFetal'] = 'error'

#Lactation P
try:
    if (Cows['Days in milking'] > 0).all():
        Cows['PMilkProd'] = 0.9*Cows['Milk production']
    else:
        Cows['PMilkProd'] = 0
except:
    Cows['PMilkProd'] = 'error'

#Growing P
try:
    if (Cows['Weight gain'] > 0).all():
        Cows['PGrowing'] = (1.2+(4.635*((Cows['mw'])**0.22)*((Cows['Body weight'])**-0.22)))*(Cows['Weight gain'] / 0.96)
    else:
        Cows['PGrowing'] = 0
except:
    Cows['PGrowing'] = 'error'

#Total P Need
Cows['Total P Need'] = Cows['PFecal'] + Cows['PUrine'] + Cows['PFetal']+ Cows['PMilkProd'] + Cows['PGrowing']

Cows

,Cow ID,Body weight,mw,Days in milking,Days of pregnancy,Breed,Milk production,Milk fat,Weight gain,CaUrine,...,CaFetal,CaMilkProd,CaGrowing,Total Ca Need,PUrine,PFecal,PFetal,PMilkProd,PGrowing,Total P Need
0,1111,500,105.737126,3,200,holstein,50,10,0.5,0.4,...,2.891023,61.0,3.63755,83.428573,1.0,24.82753,2.159901,45.0,2.340162,75.327593
1,1111,500,105.737126,3,200,holstein,50,3,0.7,0.4,...,2.891023,61.0,5.09257,84.883593,1.0,14.18841,2.159901,45.0,3.276227,65.624538


In [86]:
#Create an excel doc
Cows.to_csv(r'./FinalExcel.csv', index = False)

In [88]:
#Truncate the excel doc to only include total Ca and P
columns = ['Cow ID','Body weight','mw','Days in milking','Days of pregnancy','Breed','Milk production','Milk fat','Weight gain','Total Ca Need','Total P Need']
Cows=pd.read_csv("./FinalExcel.csv", usecols=columns)
Cows.to_csv(r'./FinalExcel.csv', index = False)

In [ ]:
##Below is just some extra code

In [ ]:
import openpyxl
wb = openpyxl.load_workbook('Cow_CaP.xlsx')
ws = wb.active
i = 0
cell_val = ''
# Finds which row is blank first
while cell_val != '':
    cell_val = ws['A' + i].value
    i += 1
# Modify Sheet, Starting With Row i
wb.save('Cow_CaP.xlsx')